# Doc2Vec
- 작성 일시: 2018-06-08
- 수정 일시: 2018-06-08
- 작성자: 부현경 (hyunkyung.boo@gmail.com)

## 1. 수집된 데이터 정제
- 수집된 데이터 합치고 DB에 데이터를 저장한다.

#### 1.1 엑셀 파일을 읽고 데이터 정제하기

In [ ]:
import pandas as pd

In [ ]:
data_df = pd.DataFrame()

num_list = [1,2]
for num in num_list:
    load_path = "D:\\naver_movie{}.xlsx".format(num)
    df = pd.read_excel(load_path, 'Sheet1')
    # data_df.append(df)
    data_df = pd.concat([data_df, df], axis = 0, ignore_index=True)
    
data_df = data_df.rename(columns = {0: 'post_id', 1: 'user_id', 2: 'movie_title', 3: 'user_review', 
                          4: 'user_rating', 5: 'post_date', 6: 'movie_genre', 7: 'running_time', 
                          8: 'release_date', 9: 'director', 10: 'actor', 11: 'movie_rating'})

In [ ]:
# 중복 제거 후 해도되고 전에 해도된다. 사람 마음...이겠지..
# release_date의 데이터 '개봉2018 . 05.01'의 형태를 '2018.05.01'로 바꿔준다.
# 약 20만건 데이터이기 때문에 이렇게 정제하는 것은 시간이 오래걸린다. (-> 필자의 컴퓨터가 느려서 그런걸지도 모르지만.. ^^;;;)
# 간단한 정제이기 때문에 엑셀의 데이터 나누기 기능을 쓰는 것이 빠르다.. (엑셀에서 20만건은 충분하다...)

num = 0
for r in data_df['release_date']:
    r = str(r).replace("개봉", "").replace(" ", "")
    print(num+1, r)
    data_df['release_date'][num] = r
    num += 1

In [ ]:
# row_data는 총 13개의 열로 구성되어 있으며 행은 195,917개이다.
print(data_df.shape)

# 중복 제거한 결과 행의 갯수가 118,773로 77,144건이 줄어들었다.
drop_duplicated_df = dataType_added_df.drop_duplicates(['post_id'])
print(drop_duplicated_df.shape)

In [ ]:
from sklearn.cross_validation import train_test_split
import numpy as np


defined_df = drop_duplicated_df.copy()
# print(defined_df.shape)

# 분할된 데이터의 각 그룹의 이름을 지정하는 함수.
def addDataType(data, col_name, typeName):
    df = pd.DataFrame(data)
    df[col_name] = typeName
    # 그룹별 형태를 확인한다. (행 갯수, 열 갯수)를 출력
    print(df.shape)
    return df

# 새로 돌릴때마다 결과가 달라진다... 유의할 것
# 학습/검증/테스트를 train, Validate, Test로 분리한다. (3:1:1 비율)
train, validate, test = np.split(defined_df.sample(frac=1), [int(.6*len(defined_df)), int(.8*len(defined_df))])
# print(train, validate, test)

Train_df = addDataType(train, 'data_type', 'Train')
Validate_df = addDataType(validate, 'data_type', 'Validate')
Test_df = addDataType(test, 'data_type', 'Test')

# 생성된 그룹들을 합친다.
defined_df = pd.concat([Train_df, Validate_df, Test_df], axis = 0, ignore_index=True)

print('레이블링한 결과', dataType_added_df.shape)
print(dataType_added_df.sample(n=5))

In [ ]:
# print(data_df.columns)
# 백업용으로 앞서 합친 데이터를 저장한다.
save_path = "D:\\defined_naver_movie_information.xlsx"
writer = pd.ExcelWriter(save_path)
dataType_added_df.to_excel(writer, 'Sheet1', header=True, index=False)
writer.save()

#### 1.2 정제된 데이터를 DB에 저장하기
- 미리 MySQL Workbench를 이용하여 테이블 및 열을 생성하였다.

In [1]:
import mysql.connector
from mysql.connector.errors import Error
import pandas as pd
import numpy as np

In [ ]:
try: 
    
    # 사용자에 맞게 설정한다.
    config = {
        'user': 'user_ name',
        'password': 'user_password',
        'host': 'localhost',
        'port': 3306,
        'database': 'db_name',
        'raise_on_warnings': True,
        'charset' : 'utf8'
    }

    conn = mysql.connector.connect(**config)
    curs = conn.cursor()
    print("커넥션 성공!")


    # 생성된 열에 결과를 저장한다.
    sql_insert_data = "INSERT INTO naver_movie_info(data_type, post_id, user_id, movie_title, user_review, \
                        user_rating, post_date, movie_genre, running_time, realese_date, director, actor, movie_score) \
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

    cnt = 0
    total = 0
    try:
                
        for row in dataType_added_df.iterrows():
               
            dt = row[1][12]
            p_id = row[1][0]
            u_id = row[1][1]
            mt = row[1][2]
            u_review = row[1][3]
            u_rating = row[1][4]
            pd = row[1][5] 
            mg = row[1][6]
            
            # 데이터 수집시 null값에 대해 nan처리가 되어
            # 이는 MySql Table에 데이터 삽입시 오류를 발생시킨다.
            # 이 때, NaN값을 ""으로 대체했다.
            if str(row[1][6]) == "nan":
                mg = ""
                rt = ""
                rd = ""
                d = ""
                a = ""
                ms = ""
                
            else :
                rt = row[1][7]
                rd = row[1][8]
                d = row[1][9]
                a = row[1][10]
                ms = row[1][11]
            
            values = (dt, p_id, u_id, mt, u_review, u_rating, pd, mg, rt, rd, d, a, ms)
            curs.execute(sql_insert_data, values)
            cnt = cnt + 1
            print(cnt)
            
            if cnt == 50:
                total += cnt
                conn.commit()
                cnt = 0

        if cnt != 50:
            print('마지막 commit 갯수: ', cnt)
            total += cnt
            conn.commit()

    # SQL execute 등이 실패했을 때
    except Exception as e:
        print(e)

# 디비 커넥션이 실패했을 때
except Exception as e:
    print(e)

finally:
    print("Insert 갯수:", total)
    curs.close()
    conn.close()